In [2]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation


import time
import tqdm
import jpype
import ast

In [3]:
# 전처리, 명사추출, 키워드추출, 불용어후처리, 닉네임추출까지 마친 데이터

import pandas as pd

folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_after_cleaning_nickname.csv')
comments_df = pd.read_csv(file_path)
sample_df = comments_df.copy()

# 특정 그룹에 대해 분석 수행
sample_df = sample_df[sample_df['Group']=='NMIXX']

# csv 파일로 저장되면서 문자열로 변형된 word_list 컬럼을 다시 리스트로 변환
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: " ".join(x))
print(len(sample_df))

# word_list가 빈 문자열인 행 제거
sample_df = sample_df[sample_df['word_list'] != '']  # 빈 문자열인 행 제거
print(len(sample_df))

#  members 컬럼을 다시 리스트로 변환
sample_df['members'] = sample_df['members'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

23511
20159


In [4]:
# TF-IDF 벡터화 (전체 데이터에 대해 벡터화 수행)
tfidf_vectorizer = TfidfVectorizer(max_features=1000)  # 최대 1000개의 단어만 사용
tfidf_matrix = tfidf_vectorizer.fit_transform(sample_df['word_list'])

# LDA 모델 학습 (전체 데이터에 대해 학습)
n_topics = n_top_words = 20  # 원하는 토픽 수 설정
lda = LatentDirichletAllocation(n_components=n_topics, 
                                max_iter=5,
                                topic_word_prior=0.1,
                                doc_topic_prior=1.0,
                                learning_method='online',
                                n_jobs=-1,
                                random_state=42)

lda.fit(tfidf_matrix)

# 각 댓글에 대해 토픽 분포 추출 (댓글별로 토픽 분포 계산)
topic_distributions = lda.transform(tfidf_matrix)

# 'topic_distribution' 열에 토픽 분포 저장
sample_df['topic_distribution'] = list(topic_distributions)

def print_top_words(model, feature_names, n_top_words=n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\n===== 토픽 {topic_idx + 1} =====")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print_top_words(lda, tfidf_feature_names, n_top_words)


===== 토픽 1 =====
노래 분위기 타이틀곡 제대로 뉴진스 케이팝 핑크 음색 인기 엔믹스 나라 가수 수록곡 최악 금발 노력 아쉬움 평가 음악 해외

===== 토픽 2 =====
엔믹스 노래 한국어 퀄리티 대중성 명곡 주년 자막 실력 올해 리스닝 매번 미래 활동 세대 집중 띵곡 안티 생일 굳이

===== 토픽 3 =====
엔믹스 권모술수 노래 조회수 비트 한국인 도입 구리 상큼 갑자기 유행 찰떡 아아 가창력 밴드 대해 개념 음원 완성 컨셉

===== 토픽 4 =====
엔믹스 노래 비주얼 파이팅 다이스 본부 뮤비 친구 지우 하루 장규진 정체 구성 영어 멤버 관심 목소리 설윤 릴리 달성

===== 토픽 5 =====
별로 중독 안무 엔믹스 별별별 중소 롤러코스터 당황 광고 버전 디스 후렴 오오 스타 노래 고생 준비 전체적 주식 쓰레기

===== 토픽 6 =====
엔믹스 지니 화이팅 규진 노래 약간 왤케 기분 커버 그녀 청량 썸네일 프로 힙합 체인지 안무가 릴리 파워 테마 콜라

===== 토픽 7 =====
응원 스타일 노래 에스파 한국 대중 수준 개인 엔믹스 반응 기획사 수고 여러 안녕 가요 인생 작곡가 넥스트레벨 멜론 더더

===== 토픽 8 =====
데뷔 소름 노래 머리 엔믹스 실험 배진솔 제왑 구려 뮤비 컨셉 스키 기획 가사도 장면 아기 이미지 영덕 이제야 유지

===== 토픽 9 =====
컴백 컨셉 쩐다 마음 엔믹스 노래 실망 중독 초반 혼란 거지 장난 원더걸스 이노 티저 대형 헤메 파트 달라 뮤비

===== 토픽 10 =====
노래 듣기 재능 소녀 회사 문제 가즈아 엔믹스 제왚 탱크 멤버 런가 음색 꽃길 실력 트렌드 외국인 아티스트 곡조 감동

===== 토픽 11 =====
진심 역대 여름 노래 보컬 뮤직비디오 벌써 엔믹스 입덕 일본 표절 표정 홍보 살짝 포인트 존예 무엇 콘서트 겨울 전환

===== 토픽 12 =====
가사 노래 제왑 걸그룹 엔믹스 여자 도대체 음방 웅장 컨셉 동요 중독 세대 엔깅이들 외국 시도 매

In [5]:

# Title별로 토픽 분포의 평균 계산
title_topic_distribution = sample_df.groupby('Title')['topic_distribution'].apply(lambda x: sum(x) / len(x))

# 정의된 토픽이름(결과 보고 이름붙임)
topic_names = {
    1: "무대 연출과 스타일",
    2: "음악 평가와 반응",
    3: "실력과 라이브 퍼포먼스",
    4: "멤버와 팬 감정",
    5: "중독성 있는 케이팝",
    6: "퀄리티와 개인 파트",
    7: "음원 성적과 대중 반응",
    8: "장르와 뮤직비디오 분석",
    9: "타이틀곡과 댄스 퍼포먼스",
    10: "컨셉과 비주얼 매력",
    11: "성과와 활동 평가",
    12: "명곡과 인기 요소",
    13: "진심과 감정 표현",
    14: "안무와 외모 평가",
    15: "데뷔와 인기 상승",
    16: "비주얼과 스타일링",
    17: "보컬 능력과 조회수",
    18: "걸그룹과 퍼포먼스 성공",
    19: "매력과 컴백 활동",
    20: "세계관과 충격 요소"
}


# Title별 상위 세 개의 토픽 분포 출력
for title, topic_dist in title_topic_distribution.items():
    print(f"\n===== {title} 그룹의 상위 3개 토픽 분포 =====")
    
    # 토픽 분포를 (토픽 번호, 확률) 형식으로 정렬
    sorted_topic_dist = sorted(enumerate(topic_dist, 1), key=lambda x: x[1], reverse=True)
    
    # 상위 3개 토픽만 출력
    for i, prob in sorted_topic_dist[:3]:
        print(f"{topic_names[i]}: {prob:.4f}")


# 첫 번째 샘플의 topic_distribution에서 최대값의 인덱스를 찾기
def get_topic_name(value):
    # 토픽 분포 중 최대값의 인덱스를 찾고, 그에 해당하는 토픽 이름 반환
    idx = np.argmax(value) + 1  # 1을 더해서 topic_names에 맞는 인덱스로 변환
    return topic_names[idx]

# 댓글의 topic_distribution에서 상위 3개의 토픽 이름을 얻기
def get_topic_names(value):
    # 토픽 분포를 내림차순으로 정렬한 후 상위 3개의 인덱스를 찾고, 그에 해당하는 토픽 이름을 반환
    top_indices = np.argsort(value)[-3:][::-1]  # 상위 3개 인덱스 (내림차순으로 정렬)
    top_topics = [topic_names[idx + 1] for idx in top_indices]  # 1을 더해서 topic_names에 맞는 인덱스로 변환
    return top_topics


sample_df['topic_distribution'] = sample_df['topic_distribution'].apply(get_topic_name)


===== DASH 그룹의 상위 3개 토픽 분포 =====
진심과 감정 표현: 0.0508
데뷔와 인기 상승: 0.0507
성과와 활동 평가: 0.0506

===== DICE 그룹의 상위 3개 토픽 분포 =====
음원 성적과 대중 반응: 0.0504
타이틀곡과 댄스 퍼포먼스: 0.0504
안무와 외모 평가: 0.0504

===== Funky Glitter Christmas 그룹의 상위 3개 토픽 분포 =====
퀄리티와 개인 파트: 0.0520
컨셉과 비주얼 매력: 0.0511
성과와 활동 평가: 0.0511

===== Love Me Like This 그룹의 상위 3개 토픽 분포 =====
타이틀곡과 댄스 퍼포먼스: 0.0512
중독성 있는 케이팝: 0.0507
진심과 감정 표현: 0.0506

===== O.O 그룹의 상위 3개 토픽 분포 =====
장르와 뮤직비디오 분석: 0.0513
명곡과 인기 요소: 0.0512
음원 성적과 대중 반응: 0.0507

===== Party O’Clock 그룹의 상위 3개 토픽 분포 =====
성과와 활동 평가: 0.0515
음원 성적과 대중 반응: 0.0507
진심과 감정 표현: 0.0506

===== Roller Coaster 그룹의 상위 3개 토픽 분포 =====
성과와 활동 평가: 0.0514
퀄리티와 개인 파트: 0.0511
보컬 능력과 조회수: 0.0509

===== Soñar (Breaker) 그룹의 상위 3개 토픽 분포 =====
실력과 라이브 퍼포먼스: 0.0520
타이틀곡과 댄스 퍼포먼스: 0.0506
세계관과 충격 요소: 0.0506

===== Young, Dumb, Stupid 그룹의 상위 3개 토픽 분포 =====
진심과 감정 표현: 0.0511
무대 연출과 스타일: 0.0510
보컬 능력과 조회수: 0.0509

===== 별별별 (See that) 그룹의 상위 3개 토픽 분포 =====
중독성 있는 케이팝: 0.0519
데뷔와 인기 상승: 0.0507
진심과 감정 표현

In [6]:
# 출력 옵션 설정: 모든 열을 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.max_rows', None)     # 모든 행 출력

sample_df[['comment','topic_distribution']].head(200)

,comment,topic_distribution
0,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,성과와 활동 평가
1,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,컨셉과 비주얼 매력
2,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,보컬 능력과 조회수
3,이야 노래 종나 멋있네,진심과 감정 표현
4,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,성과와 활동 평가
5,노래는 좋은데 ㅋㅋ 가사는 개짜치는 별별별ㅋㅋㅋㅋ,명곡과 인기 요소
6,해원이 외모체크 이후로 찐팬됨 멤버 모두 너무 착하고 예쁨 실력도 탑이네,안무와 외모 평가
7,와 진짜 설윤이랑 지우랑 비주얼 폭발하는 노래 하나 타이틀로 내자 정말 엔믹 요즘 ...,음악 평가와 반응
8,[timecode] 이어폰 까고 들으니까 무슨 연극같음,컨셉과 비주얼 매력
9,ㅈ구림,보컬 능력과 조회수


In [7]:
sample_df[['comment','topic_distribution']].head(200)

,comment,topic_distribution
0,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,성과와 활동 평가
1,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,컨셉과 비주얼 매력
2,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,보컬 능력과 조회수
3,이야 노래 종나 멋있네,진심과 감정 표현
4,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,성과와 활동 평가
5,노래는 좋은데 ㅋㅋ 가사는 개짜치는 별별별ㅋㅋㅋㅋ,명곡과 인기 요소
6,해원이 외모체크 이후로 찐팬됨 멤버 모두 너무 착하고 예쁨 실력도 탑이네,안무와 외모 평가
7,와 진짜 설윤이랑 지우랑 비주얼 폭발하는 노래 하나 타이틀로 내자 정말 엔믹 요즘 ...,음악 평가와 반응
8,[timecode] 이어폰 까고 들으니까 무슨 연극같음,컨셉과 비주얼 매력
9,ㅈ구림,보컬 능력과 조회수


In [8]:
sample_df.head()

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list,members,topic_distribution
0,NMIXX,별별별 (See that),node,UgyiK-QaarUUumBCjUJ4AaABAg,NaN,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,@lumi1216,2024-10-01T13:40:14Z,4,True,중독 진심 노래 의문 엔믹스,NaN,성과와 활동 평가
1,NMIXX,별별별 (See that),node,Ugw_6c4z7iSqIjzfgZB4AaABAg,NaN,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,@써니노,2024-10-01T09:56:20Z,1,True,기도 아쉬움 대쉬 금도 대중성 노래,NaN,컨셉과 비주얼 매력
2,NMIXX,별별별 (See that),node,UgxLdDuk-Ji3RKARe_h4AaABAg,NaN,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,@고해진-i4g,2024-10-01T05:16:15Z,4,True,본부장 엘범 천만 클릭 연락 연결 취해 피씨 뮤비 노래 뷰수 수정 재생 조치 벌써 ...,NaN,보컬 능력과 조회수
3,NMIXX,별별별 (See that),node,Ugz-Tnp-d-MLAQSXvQF4AaABAg,NaN,이야 노래 종나 멋있네,@박종한-i9u,2024-09-30T18:33:22Z,10,True,노래,NaN,진심과 감정 표현
4,NMIXX,별별별 (See that),node,UgwlfRfRmVnAsHIREut4AaABAg,NaN,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,@egi3581,2024-09-30T17:22:58Z,6,True,걸그룹 멤버 빵빵 노력 묵자 얼마나 세대 외모췍 더더 리더 해원,[해원],성과와 활동 평가


In [9]:
import gensim
from gensim import corpora

# 문서 토큰화 (예시)
documents = sample_df['word_list'].apply(lambda x: x.split())  # 'word_list'는 단어들이 공백으로 구분된 문자열로 가정

# 단어 사전(dictionary) 생성
dictionary = corpora.Dictionary(documents)

# 문서를 Bag of Words 형태로 변환 (단어: 빈도수)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# LDA 모델 학습
NUM_TOPICS = 20  # 20개의 토픽 설정
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)

# 각 토픽의 상위 단어 출력
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)


(0, '0.163*"화이팅" + 0.130*"엔믹스" + 0.062*"파트" + 0.058*"한국"')
(1, '0.122*"실력" + 0.085*"멤버" + 0.085*"보컬" + 0.077*"비주얼"')
(2, '0.165*"응원" + 0.129*"음악" + 0.046*"조회수" + 0.045*"댄스"')
(3, '0.581*"엔믹스" + 0.040*"장르" + 0.021*"파이팅" + 0.014*"반복"')
(4, '0.101*"설윤" + 0.098*"컴백" + 0.088*"규진" + 0.079*"중독"')
(5, '0.769*"노래" + 0.033*"중독" + 0.023*"정신" + 0.011*"벌써"')
(6, '0.323*"뮤비" + 0.065*"지우" + 0.041*"타이틀곡" + 0.038*"수록곡"')
(7, '0.114*"아이돌" + 0.067*"이해" + 0.059*"장난" + 0.036*"유행"')
(8, '0.478*"데뷔" + 0.032*"한국인" + 0.031*"충격" + 0.028*"회사"')
(9, '0.146*"제왑" + 0.043*"시간" + 0.042*"자꾸" + 0.040*"여름"')
(10, '0.418*"사랑해" + 0.031*"무엇" + 0.030*"영어" + 0.023*"언니"')
(11, '0.051*"지니" + 0.051*"활동" + 0.051*"매력" + 0.046*"소녀"')
(12, '0.110*"대중성" + 0.098*"분위기" + 0.088*"안무" + 0.083*"타이틀"')
(13, '0.110*"릴리" + 0.069*"진심" + 0.039*"테마" + 0.036*"음색"')
(14, '0.106*"별로" + 0.068*"누나" + 0.058*"제대로" + 0.055*"듣기"')
(15, '0.070*"마음" + 0.033*"나라" + 0.033*"호불호" + 0.032*"리듬"')
(16, '0.098*"걸그룹" + 0.082*"기대" + 0.048*"라이브" + 0.043*"무대"')
(17, 